### Char-RNN Trump-like text generation - TESTING

In [26]:
import sys
import numpy as np
import string

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.utils.data_utils import get_file
import io

N_GPU = 1 # you can experiment with more GPUs, it gets interesting with a high SEQUENCE_LEN
SEQUENCE_LEN = 10
# BATCH_SIZE = 512
BATCH_SIZE = 128
EPOCHS = 20
# HIDDEN_LAYERS_DIM = 512
HIDDEN_LAYERS_DIM = 100
# LAYER_COUNT = 4
LAYER_COUNT = 2
DROPOUT = 0.2

In [31]:
# loading the text
path = get_file('laskdhalhdla.txt', origin='https://raw.githubusercontent.com/casassg/meme_puller/master/LangModel/cleaned_captions_4.txt')
with io.open(path, encoding='utf-8') as f:
    text_train = f.read().split()
print('corpus length:', len(text_train))

# generic vocabulary
characters = sorted(list(set(text_train)))

VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)

155648/149610 [===============================] - 0s 0us/step
corpus length: 25010
vocabulary len = 5068


In [32]:
'STOP' in characters

True

In [33]:
test_model = Sequential()
for i in range(LAYER_COUNT):
    test_model.add(
            LSTM(
                HIDDEN_LAYERS_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                batch_input_shape=(1, 1, VOCABULARY_SIZE),
                stateful=True
            )
        )
test_model.add(Dense(VOCABULARY_SIZE))
test_model.add(Activation('softmax'))
test_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [34]:
test_model.load_weights(
    "./1-gpu_BS-128_2-100_dp0.20_10S_epoch20-loss5.8733_weights"
)

ValueError: Dimension 0 in both shapes must be equal, but are 5068 and 5067 for 'Assign_21' (op: 'Assign') with input shapes: [5068,400], [5067,400].

In [10]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array"""
    # from fchollet/keras
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_next_char(model, current_char, diversity=1.0):
    """Predict the next character from the current one"""
    x = np.zeros((1, 1, VOCABULARY_SIZE))
    x[:,:,characters_to_ix[current_char]] = 1
    y = model.predict(x, batch_size=1)
    next_char_ix = sample(y[0,:], temperature=diversity)
    next_char = characters[next_char_ix]
    return next_char

def generate_text(model, seed="I", count=140):
    """Generate characters from a given seed"""
    model.reset_states()
    
    next_char = predict_next_char(model, seed)
    current_char = seed

    sys.stdout.write("["+seed+"]")
    
    for i in range(count - len(seed.split())):
        next_char = predict_next_char(model, current_char, diversity=0.8)
        current_char = next_char
        sys.stdout.write(' ')
        sys.stdout.write(next_char)
    print("...\n")

In [11]:
for i in range(5):
    generate_text(
        test_model,
        seed="web_site"
    )

[web_site] humor legal thought this we band_aid to you envelope web_site went web_site hair i media comic_book a thought web_site bathing_cap good to are me kachow book_jacket a web_site envelope web_site love before it web_site just the web_site to for what know mask the him a web_site sun easter the web_site web_site my motor_scooter web_site well real when and damn help carol a balloon must envelope they why envelope web_site web_site a bed in suit web_site same in bathing_cap one make web_site comic_book at this how a web_site we president try studio_couch brainexe all common web_site i to c this work web_site thats further comic_book jute web_site binder of it notebook having he shit our to a impossible im envelope get one difficult web_site web_site ski web_site the to web_site bold crossword_puzzle be shit i are my and funny interesting...

[web_site] crutch triumphal_arch funny visit the web_site moved web_site crossword_puzzle of the meme swimming_trunks imprint with for envel